In [59]:
# pip install pypdf2
# pip install openai

In [72]:
from PyPDF2 import PdfReader

def extract_text_pdf(file_location,num_pages=None):
    # Read the file utilizing the PyPDF library.
    reader = PdfReader(file_location)
    # Obtain the total number of pages as an integer
    pages = len(reader.pages)
    extracted_text_lst = []
    if num_pages:
        total_pages = range(num_pages)
    else:
        total_pages = range(pages)
    # Extract the text from each page
    #print(len(total_pages))
    for page, _ in enumerate(total_pages):
        read = reader.pages[page]
        text = read.extract_text()
        extracted_text_lst.append(text)
    extracted_text = " ".join(extracted_text_lst)
    return extracted_text


In [73]:
import os
from openai import OpenAI
OPENAI_API_KEY = "sk-proj-xxxxxxxxxxxxxxxx" # set the right key

In [74]:
import os
def generate_response(system_prompt,content,model="gpt-4o"):
    client = OpenAI(api_key=os.getenv(OPENAI_API_KEY)) # set the right key
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": content}
        ]
        )
    return response.choices[0].message.content


In [75]:
def sort_quarter_dict(quarter_dict):
    def quarter_key(item):
        quarter, year = item[0].split('_')  # Split 'q4_2024' into 'q4' and '2024'
        quarter_num = int(quarter[1])       # Extract '4' from 'q4'
        year_num = int(year)                # Convert '2024' to integer
        return (year_num, quarter_num)      # Sort by year first, then quarter

    sorted_items = sorted(quarter_dict.items(), key=quarter_key, reverse=False)
    return dict(sorted_items)

In [76]:
def extract_quarter_info(files,prompt_loc = "prompts/extract_quarter.prompt"):
    files_info = {}
    for file_name in files:
        #passing the first few pages to extract the quarter and year
        with open(prompt_loc, "r") as prompt_file:
            system_prompt = prompt_file.read()
        content = extract_text_pdf(file_name,num_pages=3)
        result = generate_response(system_prompt,content)
        #print(file_name,result)
        files_info[result] = file_name
    return sort_quarter_dict(files_info)


In [77]:
def get_prompt(prompt_loc):
    with open(prompt_loc, "r") as prompt_file:
        prompt = prompt_file.read()
    return prompt

In [80]:

def analyze_earnings(files_info,initial_prompt_loc,multi_earnings_prompt,summarize_prompt):
    for index,file in enumerate(files_info.values()):
        if index == 0:
            print(index)
            #passing the first few pages to extract the quarter and year
            system_prompt = get_prompt(initial_prompt_loc)
            content = extract_text_pdf(file)
            quarter_response = generate_response(system_prompt = system_prompt,content = content,model="gpt-4o")
        elif (index>0)&(index<len(files_info.values())-1):
            #intermediate earnings analysis
            print('intermediate',index)
            current_earnings_content = extract_text_pdf(file)
            content = f"Previous quarter pointers:{quarter_response}. Current quarterly call transcript:{current_earnings_content}"
            system_prompt = get_prompt(multi_earnings_prompt)
            quarter_response = generate_response(system_prompt = system_prompt,content = content,model="gpt-4o")
        else:
            #final analysis
            print("final",index)
            current_earnings_content = extract_text_pdf(file)
            content = f"Previous quarter pointers:{quarter_response}. Current quarterly call transcript:{current_earnings_content}"
            system_prompt = get_prompt(summarize_prompt)
            quarter_response = generate_response(system_prompt = system_prompt,content = content,model="gpt-4o")
    return quarter_response

In [ ]:
files = ['transcripts/apr_laurus.pdf', 'transcripts/jan_laurus.pdf', 'transcripts/Jul_laurus.pdf', 'transcripts/oct_laurus.pdf']
initial_prompt_loc = 'prompts/analyse_earnings.prompt'
multi_earnings_prompt = 'prompts/analyze_multiple_earnings.prompt'
summarize_prompt = 'prompts/summarize.prompt'
files_info = extract_quarter_info(files=files)
print(files_info)
result = analyze_earnings(files_info,initial_prompt_loc,multi_earnings_prompt,summarize_prompt)

In [83]:
print(files_info)


{'q3_2024': 'transcripts/jan_laurus.pdf', 'q4_2024': 'transcripts/apr_laurus.pdf', 'q1_2025': 'transcripts/Jul_laurus.pdf', 'q2_2025': 'transcripts/oct_laurus.pdf'}


In [82]:
print(result)

# Pre-Earnings Call Analysis

### 1. **Quarterly Results vs Historical Claims Analysis**

**Order Book Confidence for Q4 FY '24:**
   - **Status:** Fully Achieved
   - **Supporting Metrics:** Positive contributions from all businesses, healthy order book with volume increase.
   - **Deviation:** None noted.

**EBITDA Margin Target >20% for FY '25:**
   - **Status:** Still Pending
   - **Updated Timeline:** FY '25
   - **Notes:** Need improved asset utilization; current margins hindered by operational deleverage.

**Improved H2 vs H1 FY '24:**
   - **Status:** Fully Achieved
   - **Supporting Metrics:** Sequential improvement due to strategic initiatives in critical divisions.

**Animal Health Revenue Peak by FY '26:**
   - **Status:** Extended to FY '27
   - **Explanation:** Delays due to technical validation requirements.
   - **Deviation:** Timeline extended by one fiscal year.

### Patterns in Historical Claims:
- **Consistently Achieved:** Revenue enhancements, order book managemen

In [3]:
from main import analyse_earnings
from utils import extract_quarter_info

files = ['transcripts/apr_laurus.pdf', 'transcripts/jan_laurus.pdf', 'transcripts/Jul_laurus.pdf', 'transcripts/oct_laurus.pdf']
initial_prompt_loc = 'prompts/analyse_earnings.prompt'
multi_earnings_prompt = 'prompts/analyze_multiple_earnings.prompt'
summarize_prompt = 'prompts/summarize.prompt'

files_info = extract_quarter_info(files=files)
print(files_info)
result = analyse_earnings(files_info,initial_prompt_loc,multi_earnings_prompt,summarize_prompt)

3
3
3
3
{'q2_2024': 'transcripts/oct_laurus.pdf', 'q3_2024': 'transcripts/jan_laurus.pdf', 'q4_2024': 'transcripts/apr_laurus.pdf', 'q1_2025': 'transcripts/Jul_laurus.pdf'}
0
18
intermediate 1
19
intermediate 2
18
final 3
16


In [4]:
print(result)

**Pre-Earnings Call Analysis**

### Quarterly Results vs. Historical Claims Analysis

#### 1. **Revenue/Margin Targets:**
   - **Management Claim**: Achieve 20% EBITDA margin for FY '25.
     - **Status**: Pending, with current margins at 14%. Management remains optimistic about improvements in H2 FY '24.
     - **Deviation**: Current margins below expectations, but management's optimistic outlook on future performance suggests confidence in achieving targets.

#### 2. **Debt Levels:**
   - **Claim**: Maintain debt below INR3,000 crore.
     - **Status**: Achieved, with current net debt at INR2,633 crore, below the threshold.

#### 3. **CDMO Performance:**
   - **Claim**: Optimize long manufacturing lead cycles.
     - **Status**: Partially achieved with ongoing improvements noted. Delays attributed to complex projects in this quarter.

#### 4. **Market Expansion Efforts:**
   - **Claim**: Broadening formulations and geographic expansion.
     - **Progress**: Mentioned focus on regulat